In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import shutil
import sys

import numpy as np
import pandas as pd
import plotly as pl

In [ ]:
sys.path.insert(0, "..")

np.random.random(20121020)

pl.offline.init_notebook_mode(connected=True)

In [ ]:
from ccal.access_gct import read_gct

df = read_gct("test_data/input.gct")

df

In [ ]:
from ccal.access_gct import write_gct

write_gct(df, "_output.gct")

os.remove("_output.gct")

In [ ]:
from ccal.access_gmt import read_gmt

df = read_gmt("test_data/c1.all.v6.1.symbols.gmt")

df

In [ ]:
from ccal.access_gmt import read_gmts

df = read_gmts(
    ("test_data/c1.all.v6.1.symbols.gmt", "test_data/h.all.v6.1.symbols.gmt")
)

df

In [ ]:
from ccal.access_gmt import write_gmt

write_gmt(df, "_output.gmt")

os.remove("_output.gmt")

In [ ]:
training_sample_x_feature = np.asarray(((0, 1),) * 8 + ((2, 3),) * 8)

training_sample_x_feature

In [ ]:
training_sample_class = np.asarray((0,) * 8 + (1,) * 8)

training_sample_class

In [ ]:
testing_sample_x_feature = training_sample_x_feature[::-1]

testing_sample_x_feature

In [ ]:
from classification.train_and_classify import train_and_classify

testing_sample_class = train_and_classify(
    training_sample_x_feature, training_sample_class, testing_sample_x_feature
)

testing_sample_class

In [ ]:
n_row = 32

n_column = 64

df = pd.DataFrame(
    np.random.random_sample(size=(n_row, n_column)),
    index=pd.Index(("Index {}".format(i) for i in range(n_row)), name="DF Row"),
    columns=pd.Index(
        ("Column {}".format(i) for i in range(n_column)), name="DF Column"
    ),
)

df

In [ ]:
ks = (2, 4, 8)

k = "K{}".format(ks[0])

In [ ]:
n_job = 3

In [ ]:
from clustering.hierarchical_consensus_cluster import hierarchical_consensus_cluster

establish_path(hierarchical_consensus_cluster.__name__, "directory")

column_cluster, column_cluster__ccc = hierarchical_consensus_cluster(
    df, ks[0], directory_path=hierarchical_consensus_cluster.__name__
)

In [ ]:
column_cluster

In [ ]:
column_cluster__ccc

In [ ]:
from clustering.hierarchical_consensus_cluster_with_ks import (
    hierarchical_consensus_cluster_with_ks,
)

establish_path(hierarchical_consensus_cluster_with_ks.__name__, "directory")

k_return = hierarchical_consensus_cluster_with_ks(
    df, ks, n_job=n_job, directory_path=hierarchical_consensus_cluster_with_ks.__name__
)

k_return[k].keys()

In [ ]:
k_return[k]["column_cluster"]

In [ ]:
k_return[k]["column_cluster.ccc"]

In [ ]:
from clustering.nmf_consensus_cluster import nmf_consensus_cluster

establish_path(nmf_consensus_cluster.__name__, "directory")

w, h, e, w_element_cluster, w_element_cluster__ccc, h_element_cluster, h_element_cluster__ccc = nmf_consensus_cluster(
    df, ks[0], directory_path=nmf_consensus_cluster.__name__
)

In [ ]:
w

In [ ]:
h

In [ ]:
e

In [ ]:
w_element_cluster

In [ ]:
w_element_cluster__ccc

In [ ]:
h_element_cluster

In [ ]:
h_element_cluster__ccc

In [ ]:
from clustering.nmf_consensus_cluster_with_ks import nmf_consensus_cluster_with_ks

establish_path(nmf_consensus_cluster_with_ks.__name__, "directory")

k_return = nmf_consensus_cluster_with_ks(
    df, ks, n_job=n_job, directory_path=nmf_consensus_cluster_with_ks.__name__
)

k_return[k].keys()

In [ ]:
k_return[k]["w"]

In [ ]:
k_return[k]["h"]

In [ ]:
k_return[k]["e"]

In [ ]:
k_return[k]["w_element_cluster"]

In [ ]:
k_return[k]["w_element_cluster.ccc"]

In [ ]:
k_return[k]["h_element_cluster"]

In [ ]:
k_return[k]["h_element_cluster.ccc"]

In [ ]:
shutil.rmtree(hierarchical_consensus_cluster.__name__)

shutil.rmtree(hierarchical_consensus_cluster_with_ks.__name__)

shutil.rmtree(nmf_consensus_cluster.__name__)

shutil.rmtree(nmf_consensus_cluster_with_ks.__name__)

In [ ]:
feature_x_sample = pd.read_table("feature_x_sample.tsv", index_col=0).iloc[:16]

feature_x_sample

In [ ]:
from context.fit_skew_t_pdf import fit_skew_t_pdf

global_n, global_location, global_scale, global_degree_of_freedom, global_shape = fit_skew_t_pdf(
    feature_x_sample.unstack().values
)

template = (
    "N={}\tLocation={:.3f}\tScale={:.3f}\tDegree of Freeedom={:.3f}\tShape={:.3f}"
)

print(
    template.format(
        global_n, global_location, global_scale, global_degree_of_freedom, global_shape
    )
)

_1d_array = feature_x_sample.iloc[0]

n, location, scale, degree_of_freedom, shape = fit_skew_t_pdf(_1d_array.values)

print(template.format(n, location, scale, degree_of_freedom, shape))

n, location, scale, degree_of_freedom, shape = fit_skew_t_pdf(
    _1d_array.values, fit_fixed_location=10, fit_fixed_scale=10
)

print(template.format(n, location, scale, degree_of_freedom, shape))

In [ ]:
from context.compute_context import compute_context

_1d_array = feature_x_sample.iloc[0].values

context_dict = compute_context(_1d_array)

context_dict.keys()

In [ ]:
context_dict["fit"]

In [ ]:
context_dict["grid"]

In [ ]:
context_dict["pdf"]

In [ ]:
context_dict["shape_pdf_reference"]

In [ ]:
context_dict["shape_context_indices"]

In [ ]:
context_dict["location_pdf_reference"]

In [ ]:
context_dict["location_context_indices"]

In [ ]:
context_dict["context_indices"]

In [ ]:
context_dict["context_indices_like_array"]

In [ ]:
from context.plot_context import plot_context

for feature, _1d_array in feature_x_sample.iloc[:8].iterrows():

    plot_context(_1d_array, title=feature)

    plot_context(
        _1d_array,
        global_location=global_location,
        global_scale=global_scale,
        global_degree_of_freedom=global_degree_of_freedom,
        global_shape=global_shape,
        title=feature,
    )

In [ ]:
from context.fit_skew_t_pdfs import fit_skew_t_pdfs

feature_skew_t_pdf_fit_parameter = fit_skew_t_pdfs(feature_x_sample)

feature_skew_t_pdf_fit_parameter

In [ ]:
from context.make_context_matrix import make_context_matrix

feature_context_matrix = make_context_matrix(
    feature_x_sample, skew_t_pdf_fit_parameter=feature_skew_t_pdf_fit_parameter
)

feature_context_matrix

In [ ]:
sample_x_feature = np.asarray(((0, 1),) * 8 + ((2, 3),) * 8)

sample_x_feature

In [ ]:
sample_class = np.asarray((0,) * 8 + (1,) * 8)

sample_class

In [ ]:
from scipy.spatial.distance import euclidean
from sklearn.svm import SVC

from cross_validation.cross_validate import cross_validate

scores = cross_validate(SVC(), sample_x_feature, sample_class, 8, euclidean)

scores

In [ ]:
point_x_dimension = np.asarray(
    (
        (-8, -8, -8),
        (-1, -1, -1),
        (0, 0, 0),
        (1, 1, 1),
        (8, 8, 8),
        (10, 10, 10),
        (16, 16, 16),
        (88, 88, 88),
    )
)

point_x_dimension

In [ ]:
from dimension_scaling.mds import mds

for n_target_dimension in (1, 2, 3):

    print("MDS n_target_dimension={}:".format(n_target_dimension))

    point_x_target_dimension = mds(
        n_target_dimension, point_x_dimension=point_x_dimension
    )

    print(point_x_target_dimension)

In [ ]:
from scipy.spatial.distance import chebyshev

point_x_target_dimension = mds(
    2, point_x_dimension=point_x_dimension, distance_function=chebyshev
)

point_x_target_dimension

In [ ]:
from scipy.spatial.distance import euclidean, pdist, squareform

point_x_target_dimension = mds(
    2, distance__point_x_point=squareform(pdist(point_x_dimension))
)

point_x_target_dimension

In [ ]:
from tables import NoSuchNodeError

In [ ]:
grch_directory_path = os.path.abspath(os.path.expanduser("."))

gff3_gz_file_path = "{}/Homo_sapiens.GRCh38.93.chr.gff3.gz".format(grch_directory_path)

assert os.path.isfile(gff3_gz_file_path), gff3_gz_file_path

In [ ]:
from feature.FeatureHDF5 import FeatureHDF5

feature_hdf5 = FeatureHDF5(gff3_gz_file_path, reset=True)

feature_hdf5

In [ ]:
feature_dict = feature_hdf5.get_features_by_name("KRAS")

feature_dict

In [ ]:
try:

    feature_hdf5.get_features_by_name("Kwat")

except KeyError as exception:

    warn("(get_features_by_name) KeyError: {}.".format(exception))

In [ ]:
feature_dict = feature_hdf5.get_features_by_region("8", 0, 800000)

feature_dict

In [ ]:
try:

    feature_hdf5.get_features_by_region("88888888", 0, 800000)

except NoSuchNodeError as exception:

    warn("(get_features_by_region) NoSuchNodeError: {}.".format(exception))

In [ ]:
from gene.select_gene_symbol import select_gene_symbol

select_gene_symbol()

In [ ]:
grch_directory_path = os.path.expanduser("~/grch")

people_directory_path = os.path.expanduser("~/people")

reference_fasta_gz_file_path = "{}/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz".format(
    grch_directory_path
)

assert os.path.isfile(reference_fasta_gz_file_path)

reference_gff3_gz_file_path = os.path.abspath(
    "{}/Homo_sapiens.GRCh38.93.chr.gff3.gz".format(grch_directory_path)
)

assert os.path.isfile(reference_gff3_gz_file_path)

vcf_gz_file_path = os.path.abspath("{}/0/genome.vcf.gz".format(people_directory_path))

assert os.path.isfile(vcf_gz_file_path)

In [ ]:
from genome.Genome import is_valid_vcf_gz

is_valid_vcf_gz(vcf_gz_file_path)

In [ ]:
from genome.Genome import Genome

genome = Genome(
    reference_fasta_gz_file_path,
    reference_gff3_gz_file_path,
    vcf_gz_file_path,
    reset=True,
)

genome

In [ ]:
genome_dict = genome.explore_genome_by_variant("rs235")

genome_dict

In [ ]:
genome_dict = genome.explore_genome_by_variant("rs88888888")

genome_dict

In [ ]:
genome_dict = genome.explore_genome_by_gene("KRAS")

genome_dict

In [ ]:
genome_dict = genome.explore_genome_by_gene("Kwat")

genome_dict

In [ ]:
genome_dict = genome.explore_genome_by_region("8", 0, 800000)

genome_dict

In [ ]:
genome_dict = genome.explore_genome_by_region("88888888", 0, 800000)

genome_dict

In [ ]:
from geo.download_and_parse_geo_data import download_and_parse_geo_data

In [ ]:
geo_dict = download_and_parse_geo_data("GSE26375")

geo_dict.keys()

In [ ]:
geo_dict["id_x_sample"]

In [ ]:
geo_dict["id_gene_symbol"]

In [ ]:
geo_dict["gene_x_sample"]

In [ ]:
geo_dict["information_x_sample"]

In [ ]:
geo_dict = download_and_parse_geo_data("GSE57820")

geo_dict.keys()

In [ ]:
geo_dict["id_x_sample"]

In [ ]:
geo_dict["id_gene_symbol"]

In [ ]:
geo_dict["gene_x_sample"]

In [ ]:
geo_dict["information_x_sample"]

In [ ]:
from sequencing_process.download_clinvar_vcf_gz import download_clinvar_vcf_gz
from sequencing_process.make_reference_genome import make_reference_genome
from sequencing_process.process_bam import (
    check_bam_using_samtools_flagstat,
    get_variants_from_bam_using_freebayes_and_multiprocess,
    get_variants_from_bam_using_strelka,
    mark_duplicates_in_bam_using_picard_markduplicates,
    sort_and_index_bam_using_samtools_sort_and_index,
)
from sequencing_process.process_fasta import faidx_fasta
from sequencing_process.process_fastq_gz import (
    align_fastq_gzs_using_bwa_mem,
    check_fastq_gzs_using_fastqc,
    trim_fastq_gzs_using_skewer,
)
from sequencing_process.process_vcf_gz import (
    annotate_vcf_gz_using_bcftools_annotate,
    annotate_vcf_gz_using_snpeff,
    filter_vcf_gz_using_bcftools_view,
    rename_chromosome_of_vcf_gz_using_bcftools_annotate,
)
from sequencing_process.support.support.path import clean_path

In [ ]:
GRCH_DIRECTORY_PATH = clean_path("~/sequencing_process_grch")

assert os.path.isdir(GRCH_DIRECTORY_PATH)

PEOPLE_DIRECTORY_PATH = clean_path("~/sequencing_process_people")

assert os.path.isdir(PEOPLE_DIRECTORY_PATH)

REGIONS = tuple("chr{}".format(i) for i in range(1, 23)) + ("chrX", "chrY", "chrM")

N_JOB = 1

MEMORY = "8G"

VARIANT_METHOD = "freebayes"

CLINVAR_VERSION = None

OVERWRITE = True

In [ ]:
try:

    FASTA_GZ_FILE_PATH = make_reference_genome(GRCH_DIRECTORY_PATH)

except FileExistsError:

    FASTA_GZ_FILE_PATH = "{}/GCA_000001405.15_GRCh38_full_plus_hs38DH-extra_analysis_set.fa.gz".format(
        GRCH_DIRECTORY_PATH
    )

    assert os.path.isfile(FASTA_GZ_FILE_PATH)

    FASTA_FILE_PATH = FASTA_GZ_FILE_PATH[:-3]

    if not os.path.isfile("{}.fai".format(FASTA_GZ_FILE_PATH)):

        faidx_fasta(FASTA_FILE_PATH)

try:

    CLINVAR_VCF_GZ_FILE_PATH = download_clinvar_vcf_gz(
        GRCH_DIRECTORY_PATH, version=CLINVAR_VERSION
    )

except FileExistsError:

    CLINVAR_VCF_GZ_FILE_PATH = "{}/{}".format(
        GRCH_DIRECTORY_PATH,
        [
            file_name
            for file_name in os.listdir(GRCH_DIRECTORY_PATH)
            if "clinvar" in file_name and file_name.endswith(".vcf.gz")
        ][0],
    )

In [ ]:
fastq_gz_0_file_path = "{}/simulation.bwa.read1.fastq.gz".format(PEOPLE_DIRECTORY_PATH)

assert os.path.isfile(fastq_gz_0_file_path)

fastq_gz_1_file_path = "{}/simulation.bwa.read2.fastq.gz".format(PEOPLE_DIRECTORY_PATH)

assert os.path.isfile(fastq_gz_1_file_path)

fastq_gz_0_trimmed_file_path, fastq_gz_1_trimmed_file_path = trim_fastq_gzs_using_skewer(
    (fastq_gz_0_file_path, fastq_gz_1_file_path),
    output_directory_path="{}/trimmed_fastq_gz".format(PEOPLE_DIRECTORY_PATH),
    n_job=N_JOB,
    overwrite=OVERWRITE,
)

check_fastq_gzs_using_fastqc(
    (
        fastq_gz_0_file_path,
        fastq_gz_1_file_path,
        fastq_gz_0_trimmed_file_path,
        fastq_gz_1_trimmed_file_path,
    ),
    n_job=N_JOB,
    overwrite=OVERWRITE,
)

In [ ]:
bam_file_path = align_fastq_gzs_using_bwa_mem(
    (fastq_gz_0_trimmed_file_path, fastq_gz_1_trimmed_file_path),
    FASTA_GZ_FILE_PATH,
    n_job=N_JOB,
    output_bam_file_path="{}/aligned.bam".format(PEOPLE_DIRECTORY_PATH),
    overwrite=OVERWRITE,
)

sorted_and_indexed_bam_file_path = sort_and_index_bam_using_samtools_sort_and_index(
    bam_file_path, remove_input_bam_file_path=True, n_job=N_JOB, overwrite=OVERWRITE
)

duplicate_removed_bam_file_path = mark_duplicates_in_bam_using_picard_markduplicates(
    sorted_and_indexed_bam_file_path,
    memory=MEMORY,
    remove_duplicates=True,
    remove_input_bam_file_path_and_its_index=True,
    n_job=N_JOB,
    output_bam_file_path="{}/duplicate_removed.bam".format(PEOPLE_DIRECTORY_PATH),
    overwrite=OVERWRITE,
)

check_bam_using_samtools_flagstat(
    duplicate_removed_bam_file_path, n_job=N_JOB, overwrite=OVERWRITE
)

In [ ]:
if VARIANT_METHOD not in ("freebayes", "strelka"):

    raise ValueError("Unknown VARIANT_METHOD: {}.".format(VARIANT_METHOD))

if VARIANT_METHOD == "freebayes":

    vcf_gz_file_path = get_variants_from_bam_using_freebayes_and_multiprocess(
        duplicate_removed_bam_file_path,
        FASTA_FILE_PATH,
        REGIONS,
        n_job=N_JOB,
        output_vcf_file_path="{}/{}.vcf".format(PEOPLE_DIRECTORY_PATH, VARIANT_METHOD),
        overwrite=OVERWRITE,
    )

    keep_filters = None

    include_expression = "10<=DP & 30<=QUAL & 10<=(QUAL/AO) & 1<=SRF & 1<=SRR & 1<=SAF & 1<=SAR & 1<=RPR & 1<=RPL"

elif VARIANT_METHOD == "strelka":

    vcf_gz_file_path = get_variants_from_bam_using_strelka(
        duplicate_removed_bam_file_path,
        FASTA_FILE_PATH,
        "{}/strelka".format(PEOPLE_DIRECTORY_PATH),
        n_job=N_JOB,
        overwrite=OVERWRITE,
    )

    keep_filters = ("PASS",)

    include_expression = "30<=QUAL"

In [ ]:
filtered_vcf_gz_file_path = filter_vcf_gz_using_bcftools_view(
    vcf_gz_file_path,
    regions=REGIONS,
    keep_filters=keep_filters,
    include_expression=include_expression,
    n_job=N_JOB,
    output_vcf_file_path="{}/filtered.vcf".format(PEOPLE_DIRECTORY_PATH),
    overwrite=OVERWRITE,
)

chromosome_renamed_vcf_gz_file_path = rename_chromosome_of_vcf_gz_using_bcftools_annotate(
    filtered_vcf_gz_file_path,
    remove_input_vcf_gz_file_path_and_its_index=True,
    n_job=N_JOB,
    output_vcf_file_path=filtered_vcf_gz_file_path.replace(
        ".vcf.gz", ".chromosome_renamed.vcf"
    ),
    overwrite=OVERWRITE,
)

snpeff_annotated_vcf_gz_file_path = annotate_vcf_gz_using_snpeff(
    chromosome_renamed_vcf_gz_file_path,
    "GRCh38.86",
    memory=MEMORY,
    remove_input_vcf_gz_file_path_and_its_index=True,
    n_job=N_JOB,
    output_vcf_file_path=chromosome_renamed_vcf_gz_file_path.replace(
        ".vcf.gz", ".snpeff.vcf"
    ),
    overwrite=OVERWRITE,
)

clinvar_annotated_vcf_gz_file_path = annotate_vcf_gz_using_bcftools_annotate(
    snpeff_annotated_vcf_gz_file_path,
    CLINVAR_VCF_GZ_FILE_PATH,
    ("--columns =ID,INFO",),
    remove_input_vcf_gz_file_path_and_its_index=True,
    n_job=N_JOB,
    output_vcf_file_path=snpeff_annotated_vcf_gz_file_path.replace(
        ".vcf.gz", ".clinvar.vcf"
    ),
    overwrite=OVERWRITE,
)

In [ ]:
w = pd.read_table("w.tsv", index_col=0)

w.columns.name = "Factor"

h = pd.read_table("h.tsv", index_col=0)

h.columns.name = "Sample"

In [ ]:
from gps_map.GPSMap import GPSMap

In [ ]:
gps_map = GPSMap(w=w.T, h=h)

In [ ]:
for w_or_h in ("w", "h"):

    gps_map.plot_gps_map(w_or_h)

In [ ]:
from gps_map.plot.plot.make_colorscale import make_colorscale

for w_or_h, elements in (("w", w.index), ("h", h.columns)):

    annotation_types = ("continuous", "categorical", "binary")

    annotation_x_element = pd.DataFrame(
        [
            np.random.random_sample(elements.size),
            np.random.randint(0, high=3, size=elements.size),
            np.random.randint(0, high=2, size=elements.size),
        ],
        index=annotation_types,
        columns=elements,
    )

    annotation_x_element.iloc[0, 0] = np.nan

    annotation_x_element.iloc[1, 1] = np.nan

    annotation_x_element.iloc[2, 2] = np.nan

    annotation_x_element.iloc[:, 3] = np.nan

    gps_map.plot_gps_map(
        w_or_h,
        annotation_x_element=annotation_x_element,
        annotation_types=annotation_types,
    )

    for annotation_type, annotation in annotation_x_element.iterrows():

        gps_map.plot_gps_map(
            w_or_h,
            annotation_x_element=annotation.to_frame().T,
            annotation_types=(annotation_type,),
            annotation_colorscale=make_colorscale(
                colors=("pink", "yellow", "purple"), plot=False
            ),
        )

    annotation_x_element = pd.DataFrame(
        [np.random.normal(size=elements.size)] * 3, columns=elements
    )

    gps_map.plot_gps_map(
        w_or_h,
        annotation_x_element=annotation_x_element,
        annotation_std_maxs=(0.1, 1, 3),
    )

    gps_map.plot_gps_map(
        w_or_h,
        annotation_x_element=annotation_x_element.iloc[:1],
        annotation_ranges=((0, 8),),
    )

In [ ]:
from gps_map.plot.plot.make_random_color import make_random_color

for w_or_h, element_labels in (
    ("w", w.apply(np.argmax, axis=1).str[1:].astype(int)),
    ("h", h.apply(np.argmax).str[1:].astype(int)),
):

    gps_map.set_element_labels(w_or_h, element_labels)

    gps_map.plot_gps_map(w_or_h)

In [ ]:
for w_or_h, w_or_h_matrix in (("w", w.T), ("h", h)):

    predicted_element_labels = gps_map.predict(w_or_h, w_or_h_matrix.iloc[:, :8])

    print(predicted_element_labels)

In [ ]:
for w_or_h in ("w", "h"):

    gps_map.anneal_node_and_element_positions(
        w_or_h, n_iteration=64, initial_temperature=0.01
    )

    gps_map.plot_gps_map(w_or_h)

In [ ]:
h = pd.read_table("kras_gps_map/nmf_k9_h.tsv", index_col=0)

# https://github.com/UCSD-CCAL/onco_gps_paper_analysis/blob/master/code/8%20Define%20global%20cellular%20labels%20and%20make%20global%20Onco-GPS%20map.ipynb

h = h.apply(lambda row: (row - row.mean()) / row.std(), axis=1)

h = h.clip(lower=-3, upper=3)

h = h.apply(lambda row: (row - row.min()) / (row.max() - row.min()), axis=1)

In [ ]:
# https://github.com/KwatME/gps_map/blob/e932c53e1d679a68c224cf62284fb9209235962f/onco_gps/GPSMap.py

h = h.apply(lambda row: (row - row.mean()) / row.std(), axis=1)

h = h.clip(lower=-3, upper=3)

h = h.apply(lambda row: (row - row.min()) / (row.max() - row.min()), axis=1)

# Euclidean distance is used instead in https://github.com/KwatME/gps_map/blob/e932c53e1d679a68c224cf62284fb9209235962f/onco_gps/GPSMap.py

gps_map = GPSMap(h=h, h_pull_power=2.4, plot=False)

In [ ]:
h_element_labels = pd.read_table("kras_gps_map/hcc__k_x_h_column.tsv", index_col=0).loc[
    "K15"
]

gps_map.set_element_labels(
    "h",
    h_element_labels,
    label_colors=(
        "#e74c3c",
        "#ffd700",
        "#4b0082",
        "#993300",
        "#4169e1",
        "#90ee90",
        "#f4bd60",
        "#8b008b",
        "#fa8072",
        "#b0e0e6",
        "#20d9ba",
        "#da70d6",
        "#d2691e",
        "#dc143c",
        "#2e8b57",
    ),
    bandwidth_factor=5.6,
)

gps_map.plot_gps_map("h")

In [ ]:
predicted_element_labels = gps_map.predict("h", h)

predicted_element_labels

In [ ]:
from gps_map.access_gps_map import dump_gps_map

dump_gps_map(gps_map, "gps_map.pickle.gz")

In [ ]:
from gps_map.access_gps_map import load_gps_map

gps_map = load_gps_map("gps_map.pickle.gz")

gps_map.plot_gps_map("h")

os.remove("gps_map.pickle.gz")

In [ ]:
n_gene = 80

all_genes = pd.read_table("genes.txt", squeeze=True)

genes = np.random.choice(all_genes, size=n_gene, replace=False)

genes

In [ ]:
n_sample = 8

gene_x_sample = pd.DataFrame(
    np.random.random_sample(size=(n_gene, n_sample)),
    index=genes,
    columns=("Sample {}".format(i) for i in range(n_sample)),
)

gene_x_sample

In [ ]:
n_gene_set = 8

gene_sets = []

for i in range(n_gene_set):

    gene_sets.append(np.random.choice(genes, size=(i + 1) * 8, replace=False))

gene_sets = pd.DataFrame(
    gene_sets, index=tuple("Gene Set {}".format(i) for i in range(n_gene_set))
)

gene_sets

In [ ]:
from gsea.single_sample_gsea import single_sample_gsea

score = single_sample_gsea(gene_x_sample.iloc[:, 0] - 0.5, gene_sets.iloc[0])

score

In [ ]:
from gsea.nd_array.nd_array.normalize_nd_array import normalize_nd_array

for statistic in ("ks", "auc"):

    gene_score = gene_x_sample.iloc[:, 0]

    for normalization_method in (None, "0-1", "-0-"):

        if normalization_method is not None:

            gene_score = pd.Series(
                normalize_nd_array(gene_score, None, normalization_method),
                name=gene_score.name,
                index=gene_score.index,
            )

        single_sample_gsea(
            gene_score,
            gene_sets.iloc[0],
            statistic=statistic,
            title="statistic={} & normalization_method={}".format(
                statistic, normalization_method
            ),
        )

In [ ]:
from gsea.single_sample_gseas import single_sample_gseas

score__gene_set_x_sample = single_sample_gseas(gene_x_sample, gene_sets, n_job=2)

score__gene_set_x_sample

In [ ]:
from scipy.stats import pearsonr

from gsea.gsea import gsea

n_sample_per_phenotype = n_sample // 2

phenotypes = np.array((0,) * n_sample_per_phenotype + (1,) * n_sample_per_phenotype)


def function(x, y):

    return pearsonr(x, y)[0]


score_p_value = gsea(
    gene_x_sample,
    phenotypes,
    gene_sets.iloc[0, :],
    function,
    statistic="ks",
    n_permutation=10,
    permuting="gene",
    gene_score_name="Correlation",
)

score_p_value

# Make gene x cellline

In [ ]:
# Load CCLE mutation-amplification-deletion x cellline
mutation_amplification_deletion_x_cellline = gct.read_gct(
    "../data/ccle/mutation__gene_x_ccle_cellline.gct"
)

print(
    "mutation_amplification_deletion_x_cellline.shape: {}".format(
        mutation_amplification_deletion_x_cellline.shape
    )
)

# Keep only mutations (*_MUT)
mutation_x_cellline = mutation_amplification_deletion_x_cellline.loc[
    mutation_amplification_deletion_x_cellline.index.str.endswith("MUT")
]

print("mutation_x_cellline.shape: {}".format(mutation_x_cellline.shape))

# Remove '_MUT' suffix
mutation_x_cellline.index = [l[0] for l in mutation_x_cellline.index.str.split(sep="_")]

# Select genes

In [ ]:
# Read Foundation One genes
foundation_one_genes = (
    gmt.read_gmts("../data/gene_sets/cancer_gene_sets/cancer_gene_sets.gmt")
    .loc["FoundationOne"]
    .dropna()
    .tolist()
)

print("N Foundation One genes: {}".format(len(foundation_one_genes)))

# Keep only mutations in the Foundation One genes
selected_mutation_x_cellline = mutation_x_cellline.loc[
    [i in foundation_one_genes for i in mutation_x_cellline.index]
]

print("selected_mutation_x_cellline: {}".format(selected_mutation_x_cellline.shape))

# Make compound-sensitivity x cellline

In [ ]:
# Read CCLE compound-sensitivity x cellline
compound_sensitivity_x_cellline = gct.read_gct(
    "../data/ccle/ctd2__compound_x_ccle_cellline.gct"
)

# Flip value signs because lower the viability score, better the response
compound_sensitivity_x_cellline *= -1

compound_sensitivity_x_cellline = pd.DataFrame(
    a2d.normalize(compound_sensitivity_x_cellline.values, "0-1"),
    index=compound_sensitivity_x_cellline.index,
    columns=compound_sensitivity_x_cellline.columns,
)

plot.plot_heatmap(
    compound_sensitivity_x_cellline,
    title="compound_sensitivity_x_cellline {}".format(
        compound_sensitivity_x_cellline.shape
    ),
    xlabel="Cellline",
    ylabel="Compound",
)

# Get cell lines common in selected-mutation x cellline and compound-sensitivity x cellline
common_celllines = (
    selected_mutation_x_cellline.columns & compound_sensitivity_x_cellline.columns
).sort_values()

print("common_celllines.size: {}".format(common_celllines.size))

# Match selected mutations to compound sensitivity

In [ ]:
compound = "PLX-4720"
print("Matching mutations to {} ...".format(compound))

compound_sensitivity = compound_sensitivity_x_cellline.loc[
    compound, common_celllines
].dropna()
print("\tcompound_sensitivity.size: {}".format(compound_sensitivity.size))

print()
match_result = make_match_panel(
    compound_sensitivity,
    selected_mutation_x_cellline.loc[:, compound_sensitivity.index],
    n_jobs=4,
    n_features=10,
    n_samplings=0,
    n_permutations=0,
    features_type="binary",
)

# Infer compound response from mutations

## Set up variables

In [ ]:
mutation_1 = "BRAF"
mutation_2 = "KRAS"

print("Mutaiton 1: {}".format(mutation_1))
print("Mutaiton 2: {}".format(mutation_2))

m1 = np.array(selected_mutation_x_cellline.loc[mutation_1, compound_sensitivity.index])
m2 = np.array(selected_mutation_x_cellline.loc[mutation_2, compound_sensitivity.index])
r = np.array(compound_sensitivity)
rb = (0.6 < r).astype(int)

plot.plot_distribution(m1, title="{} Distribution".format(mutation_1))
plot.plot_distribution(m2, title="{} Distribution".format(mutation_2))
plot.plot_distribution(r, title="{} Distribution".format(compound))
plot.plot_distribution(rb, title="{} (binarized) Distribution".format(compound))

grid_size = 8

## Compute P(R | Mi)

In [ ]:
p_rb__m1, p_rb1__m1 = infer(
    [m1, rb], grid_size=grid_size, target=1, variable_names=[mutation_1, compound]
)

p_rb__m2, p_rb1__m2 = infer(
    [m2, rb], grid_size=grid_size, target=1, variable_names=[mutation_2, compound]
)

## Compute P(R | M1, M2)

In [ ]:
p_rb__m1m2, p_rb1__m1m2 = infer_assuming_independence(
    [m1, m2, rb],
    grid_size=grid_size,
    target=1,
    variable_names=[mutation_1, mutation_2, compound],
)

p_rb__m1m2, p_rb1__m1m2 = infer(
    [m1, m2, rb],
    grid_size=grid_size,
    target=1,
    variable_names=[mutation_1, mutation_2, compound],
)

## Make Bayesian nomograms

In [ ]:
p_rb1 = rb.sum() / rb.size

print(p_rb1)

In [ ]:
plot_bayesian_nomogram(
    [mutation_1, mutation_2],
    [m1, m2],
    [p_rb__m1, p_rb__m2],
    p_rb1,
    1 - p_rb1,
    n_lors_marks=2,
    file_path="../results/nomogram.png",
)

plot_bayesian_nomogram(
    [mutation_1, mutation_2],
    [m1, m2],
    [p_rb__m1, p_rb__m2],
    p_rb1,
    1 - p_rb1,
    sample=[1, 1],
    n_lors_marks=2,
    file_path="../results/nomogram_with_sample.png",
)

In [ ]:
from information.compute_entropy import compute_entropy

for arrar_1d in (
    np.random.random_sample(8),
    np.random.random_sample(16),
    np.random.random_sample(32),
    np.random.random_sample(64),
    np.random.random_sample(128),
    np.random.random_sample(256),
    np.random.random_sample(512),
    np.random.random_sample(1024),
    np.random.random_sample(1000000),
    np.asarray((0,) * 100),
    np.asarray((1,) * 100),
    np.asarray((1,) * 1000),
    np.asarray((1,) * 10000),
    np.asarray((1,) * 100000),
    np.asarray((1,) * 1000000),
):

    entropy = compute_entropy(arrar_1d)

    print(entropy)

In [ ]:
from information.compute_information_coefficient import compute_information_coefficient
from information.compute_information_distance import compute_information_distance

for x, y in (
    (np.asarray((0,) * 8),) * 2,
    (np.asarray((1,) * 8),) * 2,
    (np.asarray(range(8)), np.asarray(range(8))[::-1]),
    (np.asarray((0, 1, 2, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2, 1, 0)),) * 2,
    (np.asarray((8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8)),) * 2,
    (np.asarray((0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)),) * 2,
    (
        np.asarray((1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1)),
        np.asarray((0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0)),
    ),
):

    print(0)

    print(x)

    print(y)

    information_coefficient = compute_information_coefficient(x, y)

    information_distance = compute_information_distance(x, y)

    print(
        "Information Coefficient = {:.3f}\tInformation Distance = {:.3f}".format(
            information_coefficient, information_distance
        )
    )

In [ ]:
n = 3

x = np.asarray((0,) * n + (1,) * n + (2,) * n)

print(x)

for factor in (0.1, 0.5, 1, 2, 10, 100):

    y = x * factor

    print()

    print(y)

    ic = compute_information_coefficient(x, y)

    print(ic)

In [ ]:
from kernel_density.compute_bandwidths import compute_bandwidths

coordiante_1d = np.asarray(range(16))

for n_dimension in (1, 2, 8):

    bandwidths = compute_bandwidths(
        (coordiante_1d,) * n_dimension, ("c",) * n_dimension
    )

    print(bandwidths)

In [ ]:
from kernel_density.estimate_kernel_density import estimate_kernel_density

for i, j in (
    (np.asarray(range(8)), np.asarray(range(8))),
    (np.asarray((0, 1, 2, 3)), np.asarray((1, 1, 0, 0))),
    (np.asarray((0, 1, 2, 3)), np.asarray((0, 0, 1, 1))),
    (np.asarray((1, 1, 0, 0)), np.asarray((0, 1, 2, 3))),
    (np.asarray((0, 0, 1, 1)), np.asarray((0, 1, 2, 3))),
):

    ij = estimate_kernel_density((i, j))

    pl.offline.iplot(
        dict(
            layout=dict(
                width=400, height=400, xaxis=dict(title="j"), yaxis=dict(title="i")
            ),
            data=[dict(type="heatmap", z=ij[::-1])],
        )
    )

In [ ]:
coordiante_1d = np.asarray(range(8))

n_dimension = 3

ijk = estimate_kernel_density((coordiante_1d,) * n_dimension)

ijk

In [ ]:
from linear_model.correlate import correlate

for x, y in (
    (np.random.random_sample(16),) * 2,
    (np.random.random_sample(16), np.random.random_sample(16)),
    (np.asarray(range(16)), np.asarray(range(16, 0, -1))),
):

    r2, p_value = correlate(x, y, n_permutation=100, plot=True)

In [ ]:
from match.make_match_panel import make_match_panel

In [ ]:
_2d_array = np.asarray(((0, np.nan), (0, np.nan)))

features = pd.DataFrame(
    _2d_array,
    index=("Index {}".format(i) for i in range(_2d_array.shape[0])),
    columns=("Column {}".format(i) for i in range(_2d_array.shape[1])),
)

target = pd.Series(
    np.random.normal(loc=0, scale=1, size=features.columns.size), index=features.columns
)

score_moe_p_value_fdr = make_match_panel(
    target, features, n_sampling=1, n_permutation=1
)

score_moe_p_value_fdr

In [ ]:
n_index = 8

n_column = 64

features = pd.DataFrame(
    np.random.random_sample(size=(n_index, n_column)),
    index=("Index {}".format(i) for i in range(n_index)),
    columns=("Column {}".format(i) for i in range(n_column)),
)

target = pd.Series(
    np.random.normal(loc=0, scale=1, size=n_column),
    name="TARGET",
    index=features.columns,
)

score_moe_p_value_fdr = make_match_panel(
    target, features, n_sampling=2, n_permutation=1
)

score_moe_p_value_fdr

In [ ]:
for plot_only_sign in ("-", "+"):

    score_moe_p_value_fdr = make_match_panel(
        target, features, n_sampling=2, n_permutation=1, plot_only_sign=plot_only_sign
    )

In [ ]:
for n_index, n_column in (
    (8, 2),
    (8, 4),
    (8, 16),
    (8, 32),
    (8, 160),
    (8, 320),
    (8, 1600),
    (8, 3200),
    (1, 8),
    (2, 8),
    (4, 8),
    (16, 8),
    (32, 8),
    (64, 8),
    (128, 8),
    (320, 8),
    (640, 8),
):

    target = pd.Series(
        np.random.random_sample(size=n_column),
        index=("Column {}".format(i) for i in range(n_column)),
    )

    features = pd.DataFrame(
        np.random.random_sample(size=(n_index, n_column)),
        index=("Index {}".format(i) for i in range(n_index)),
        columns=("Column {}".format(i) for i in range(n_column)),
    )

    score_moe_p_value_fdr = pd.DataFrame(
        np.random.random_sample(size=(features.shape[0], 4)),
        index=("Index {}".format(i) for i in range(n_index)),
        columns=("Score", "0.95 MoE", "P-Value", "FDR"),
    )

    score_moe_p_value_fdr = make_match_panel(
        target,
        features,
        score_moe_p_value_fdr=score_moe_p_value_fdr,
        extreme_feature_threshold=None,
        title="{}-x-{} Match Panel".format(n_index, n_column),
    )

In [ ]:
target = [0, 2]

target = pd.Series(target, index=("Sample {}".format(i) for i in range(len(target))))

n_row = 8

features = pd.DataFrame(
    np.random.random_sample(size=(n_row, target.size)),
    index=("Index {}".format(i) for i in range(n_row)),
    columns=target.index,
)

for random_seed in range(3):

    score_moe_p_value_fdr = make_match_panel(
        target,
        features,
        n_sampling=1,
        n_permutation=1,
        random_seed=random_seed,
        target_type="binary",
    )

In [ ]:
target = [0, 2, 8, 0]

target = pd.Series(target, index=("Sample {}".format(i) for i in range(len(target))))

n_row = 8

features = pd.DataFrame(
    np.random.random_sample(size=(n_row, target.size)),
    index=("Index {}".format(i) for i in range(n_row)),
    columns=target.index,
)

score_moe_p_value_fdr = make_match_panel(
    target,
    features,
    n_sampling=2,
    n_permutation=1,
    target_ascending=None,
    target_type="categorical",
)

score_moe_p_value_fdr

In [ ]:
target = pd.read_table("target_0.tsv", index_col=0, squeeze=True)

features = pd.read_table("features_0.tsv", index_col=0)

score_moe_p_value_fdr = make_match_panel(
    target, features, n_sampling=2, n_permutation=1, target_type="binary"
)

score_moe_p_value_fdr

In [ ]:
score_moe_p_value_fdr = make_match_panel(
    features.iloc[0], features.iloc[:1], n_sampling=2, n_permutation=1, plot_std=3
)

score_moe_p_value_fdr

In [ ]:
score_moe_p_value_fdr = make_match_panel(
    features.iloc[0], features.iloc[:2], n_sampling=2, n_permutation=1, plot_std=3
)

score_moe_p_value_fdr

In [ ]:
n_index = 24

n_column = 16

features = pd.DataFrame(
    np.random.random_sample(size=(n_index, n_column)),
    index=("Index {}".format(i) for i in range(n_index)),
    columns=("Column {}".format(i) for i in range(n_column)),
)

features.iloc[:2, -2:] = np.nan

for target in (
    pd.Series(("Aa",) * 4 + ("Bb",) * 4 + ("Cc",) * 4 + ("Aa",) * 4),
    pd.Series((0,) * 8 + (2,) * 8),
    pd.Series((0,) * 1 + (1,) * 2 + (2,) * 13),
    pd.Series((0,) * 2 + (1,) * 2 + (3,) * 12),
):

    target.index = ("Column {}".format(i) for i in range(n_column))

    score_moe_p_value_fdr = make_match_panel(
        target,
        features,
        target_ascending=None,
        n_sampling=2,
        n_permutation=1,
        score_ascending=True,
        target_type="categorical",
    )

    print(score_moe_p_value_fdr)

In [ ]:
n_index = 8

n_column = 16

target = pd.Series(
    np.random.random_sample(size=n_column + 1),
    index=("Column {}".format(i) for i in range(n_column + 1)),
)

features = pd.DataFrame(
    np.random.random_sample(size=(n_index, n_column)),
    index=("ABCDEFGHIJKLMNOPQRSTUVWXYZ {}".format(i) for i in range(n_index)),
    columns=("Column {}".format(i) for i in range(n_column)),
)

features.iloc[0] = 0

features.loc[
    np.random.choice(features.index, size=features.index.size // 8 * 7, replace=False),
    np.random.choice(
        features.columns, size=features.columns.size // 8 * 7, replace=False
    ),
] = np.nan

score_moe_p_value_fdr = make_match_panel(
    target, features, n_sampling=2, n_permutation=1
)

score_moe_p_value_fdr

In [ ]:
n_index = 8

n_column = 64

target = pd.Series(
    np.random.random_sample(size=n_column),
    index=("Column {}".format(i) for i in range(n_column)),
)

In [ ]:
features = pd.DataFrame(
    np.random.random_sample(size=(n_index, n_column)),
    index=("Index {}".format(i) for i in range(n_index)),
    columns=("Column {}".format(i) for i in range(n_column)),
)

score_moe_p_value_fdr = make_match_panel(target, features)

score_moe_p_value_fdr

In [ ]:
features = pd.DataFrame(
    np.random.random_integers(0, 8, size=(n_index, n_column)),
    index=("Index {}".format(i) for i in range(n_index)),
    columns=("Column {}".format(i) for i in range(n_column)),
)

score_moe_p_value_fdr = make_match_panel(target, features, features_type="categorical")

score_moe_p_value_fdr

In [ ]:
features = pd.DataFrame(
    np.random.random_integers(0, 1, size=(n_index, n_column)),
    index=("Index {}".format(i) for i in range(n_index)),
    columns=("Column {}".format(i) for i in range(n_column)),
)

score_moe_p_value_fdr = make_match_panel(target, features, features_type="binary")

score_moe_p_value_fdr

In [ ]:
from match.make_summary_match_panel import make_summary_match_panel

In [ ]:
n_index = 4

n_column = 8

target = pd.Series(
    np.random.random_sample(size=n_column),
    name="TARGET",
    index=("Column {}".format(i) for i in range(n_column)),
)

features_continuous = pd.DataFrame(
    np.random.random_sample(size=(n_index, n_column)),
    index=("Continuous {}".format(i) for i in range(n_index)),
    columns=target.index,
)

features_categorical = pd.DataFrame(
    np.random.randint(0, 10, size=(n_index, n_column)),
    index=("Categorical {}".format(i) for i in range(n_index)),
    columns=target.index,
)

features_binary = pd.DataFrame(
    np.random.randint(0, 2, size=(n_index, n_column)),
    index=("Binary {}".format(i) for i in range(n_index)),
    columns=target.index,
)

# features_continuous.iloc[:, 0] = np.nan

# features_categorical.iloc[:, 1] = np.nan

# features_binary.iloc[:, 2] = np.nan

features_continuous.drop(target.index[-1], axis=1, inplace=True)

features_categorical.drop(target.index[-2], axis=1, inplace=True)

features_binary.drop(target.index[-5:-3], axis=1, inplace=True)

In [ ]:
data_dicts = {
    "Continuous": {
        "df": features_continuous,
        "data_type": "continuous",
        "emphasis": "low",
    },
    "Categorical": {
        "df": features_categorical,
        "data_type": "categorical",
        "emphasis": "high",
    },
    "Binary": {"df": features_binary, "data_type": "binary", "emphasis": "low"},
}

In [ ]:
score_moe_p_value_fdr = make_match_panel(
    target,
    pd.concat((data_dict["df"] for data_dict in data_dicts.values())),
    n_sampling=1,
    n_permutation=1,
    plot_std=3,
)

score_moe_p_value_fdr

In [ ]:
make_summary_match_panel(target, data_dicts, score_moe_p_value_fdr)

In [ ]:
make_summary_match_panel(
    target,
    data_dicts,
    score_moe_p_value_fdr,
    plot_only_columns_shared_by_target_and_all_features=True,
)

In [ ]:
features_na = pd.DataFrame(
    np.nan, index=features_continuous.index, columns=features_continuous.columns
)

make_summary_match_panel(
    target,
    {"NA": {"df": features_na, "data_type": "continuous", "emphasis": "high"}},
    score_moe_p_value_fdr,
)

In [ ]:
from match.make_comparison_panel import make_comparison_panel

In [ ]:
_2d_array_0 = np.random.random_sample(size=(8, 2))

_2d_array_1 = np.random.random_sample(size=(8, 4))

comparison = make_comparison_panel(
    _2d_array_0, _2d_array_1, name_0="2D Array 0", name_1="2D Array 1"
)

comparison

In [ ]:
n_column = 16

df_0 = pd.DataFrame(
    np.random.random_sample(size=(2, n_column)),
    index=("DF 0 Index {}".format(i) for i in range(2)),
    columns=("DF 0 Column {}".format(i) for i in range(n_column)),
)

df_1 = pd.DataFrame(
    np.random.random_sample(size=(8, n_column)),
    index=("DF 1 Index {}".format(i) for i in range(8)),
    columns=("DF 1 Column {}".format(i) for i in range(n_column)),
)

comparison = make_comparison_panel(df_0, df_1, axis=1, name_0="DF 0", name_1="DF 1")

comparison

In [ ]:
from matrix_factorization.nmf_by_multiple_V_and_H import nmf_by_multiple_V_and_H
from matrix_factorization.nmf_by_multiplicative_update import (
    nmf_by_multiplicative_update,
)
from matrix_factorization.nmf_by_sklearn import nmf_by_sklearn

In [ ]:
def plot_R_norms(R_norms, title=None):

    if title is None:

        title = "NMF Convergence"

    pl.offline.iplot(
        dict(
            layout=dict(
                width=640,
                height=640,
                title=title,
                xaxis=dict(title="Iteration"),
                yaxis=dict(title="Residual Matrix Norm"),
            ),
            data=[
                dict(type="scatter", name=i, x=list(range(R_norms_.size)), y=R_norms_)
                for i, R_norms_ in enumerate(R_norms)
            ],
        )
    )

In [ ]:
def plot_Ws_and_Hs(Ws, Hs):

    for i, W in enumerate(Ws):

        pl.offline.iplot(
            dict(
                layout=dict(
                    width=320,
                    height=640,
                    title="W {}".format(i),
                    xaxis=dict(title="k"),
                    yaxis=dict(title="m"),
                ),
                data=[
                    dict(
                        type="heatmap", z=W[::-1], colorscale="Picnic", showscale=False
                    )
                ],
            )
        )

    for i, H in enumerate(Hs):

        pl.offline.iplot(
            dict(
                layout=dict(
                    width=640,
                    height=320,
                    title="H {}".format(i),
                    xaxis=dict(title="n"),
                    yaxis=dict(title="k"),
                ),
                data=[
                    dict(
                        type="heatmap", z=H[::-1], colorscale="Picnic", showscale=False
                    )
                ],
            )
        )

In [ ]:
m = 160

n = 80

V = np.random.random_sample(size=(m, n))

V += abs(V.min())

Vs = (V, V * 10)

for V in Vs:

    print(V.shape)

In [ ]:
k = 2

n_iteration = int(1e3)

random_seed = 20121020

In [ ]:
individual_R_norms = []

for V in Vs:

    W, H, R_norms = nmf_by_multiplicative_update(
        V, k, n_iteration=n_iteration, random_seed=random_seed
    )

    plot_Ws_and_Hs((W,), (H,))

    print("R norm (multiplicative_update): {:.2f}".format(R_norms[-1]))

    individual_R_norms.append(R_norms)

    W_by_sklean, H_by_sklean, R_by_sklean = nmf_by_sklearn(
        V, k, n_iteration=n_iteration, random_seed=random_seed
    )

    plot_Ws_and_Hs((W_by_sklean,), (H_by_sklean,))

    print("R norm (sklean): {:.2f}".format(R_by_sklean))

plot_R_norms(individual_R_norms, title="NMF Independently")

In [ ]:
W, Hs, R_norms = nmf_by_multiple_V_and_H(
    Vs, k, n_iteration=n_iteration, random_seed=random_seed
)

plot_Ws_and_Hs((W,), Hs)

print(
    "R norm (multiple_V_and_H): {}".format(
        ", ".join("{:.2f}".format(float_) for float_ in R_norms[:, -1])
    )
)

plot_R_norms(R_norms, title="NMF Together")

In [ ]:
from nd_array.apply_function_on_2_1d_arrays import apply_function_on_2_1d_arrays

In [ ]:
_1d_array_0 = np.asarray((np.nan, -1, 0, 2, 4, 8))

_1d_array_1 = np.asarray((8, np.inf, -np.inf, 4, 2, -1))

In [ ]:
def function(_1d_array_0, _1d_array_1):

    return _1d_array_0 + _1d_array_1

In [ ]:
try:

    function_return = apply_function_on_2_1d_arrays(_1d_array_0, _1d_array_1, function)

    print(function_return)

except ValueError as exception:

    print(exception)

In [ ]:
function_return = apply_function_on_2_1d_arrays(
    _1d_array_0, _1d_array_1, function, raise_for_bad=False
)

function_return

In [ ]:
try:

    function_return = apply_function_on_2_1d_arrays(
        _1d_array_0, _1d_array_1, function, raise_for_bad=False, use_only_good=False
    )

    print(function_return)

except ValueError as exception:

    print(exception)

In [ ]:
try:

    function_return = apply_function_on_2_1d_arrays(
        _1d_array_0, _1d_array_1, function, n_required=4, raise_for_bad=False
    )

    print(function_return)

except ValueError as exception:

    print(exception)

In [ ]:
function_return = apply_function_on_2_1d_arrays(
    _1d_array_0,
    _1d_array_1,
    function,
    n_required=4,
    raise_for_n_less_than_required=False,
    raise_for_bad=False,
)

function_return

In [ ]:
from scipy.spatial.distance import euclidean

In [ ]:
n_permutation = 100

for p_value_direction in ("less", "great"):

    print()

    print(p_value_direction)

    function_return = apply_function_on_2_1d_arrays(
        _1d_array_0,
        _1d_array_1,
        euclidean,
        n_permutation=n_permutation,
        p_value_direction=p_value_direction,
        raise_for_bad=False,
    )

    print(function_return)

In [ ]:
from nd_array.apply_function_on_2_2d_arrays_slices import (
    apply_function_on_2_2d_arrays_slices,
)

_2d_array_0 = np.random.random_sample(size=(8, 2))

_2d_array_1 = np.random.random_sample(size=(8, 4))

In [ ]:
function_return = apply_function_on_2_2d_arrays_slices(
    _2d_array_0, _2d_array_1, euclidean, 0
)

function_return

In [ ]:
try:

    function_return = apply_function_on_2_2d_arrays_slices(
        _2d_array_0, _2d_array_1, euclidean, 1
    )

    print(function_return)

except ValueError as exception:

    print(exception)

In [ ]:
from nd_array.check_nd_array_for_bad import check_nd_array_for_bad

for nd_array in (
    np.asarray((np.nan, 0, 1)),
    np.asarray((0, np.inf, 1)),
    np.asarray((0, 1, -np.inf)),
    np.asarray((np.nan, np.inf, -np.inf)),
):

    print()

    print(nd_array)

    try:

        is_bad = check_nd_array_for_bad(nd_array)

        print(is_bad)

    except ValueError as exception:

        print(exception)

    is_bad = check_nd_array_for_bad(nd_array, raise_for_bad=False)

    print(is_bad)

In [ ]:
from nd_array.clip_nd_array_by_standard_deviation import (
    clip_nd_array_by_standard_deviation,
)

nd_array = np.random.normal(size=16)

nd_array.max(), nd_array.min()

In [ ]:
nd_array_clipped_by_standard_deviation = clip_nd_array_by_standard_deviation(
    nd_array, 1
)

nd_array_clipped_by_standard_deviation.max(), nd_array_clipped_by_standard_deviation.min()

In [ ]:
nd_array[0] = np.nan

nd_array[1] = np.inf

nd_array[2] = -np.inf

nd_array_clipped_by_standard_deviation = clip_nd_array_by_standard_deviation(
    nd_array, 1, raise_for_bad=False
)

nd_array_clipped_by_standard_deviation.max(), nd_array_clipped_by_standard_deviation.min()

In [ ]:
from nd_array.cluster_2d_array_slices import cluster_2d_array_slices

_2d_array = np.asarray(
    ((0, 0), (-2, 2), (-4, 4), (-8, 8), (0, 0), (-4, 4), (-8, 8), (-2, 2))
)

In [ ]:
indices = cluster_2d_array_slices(_2d_array, 0)

indices

In [ ]:
indices = cluster_2d_array_slices(
    _2d_array, 0, groups=np.asarray((1,) * 4 + (0,) * 2 + (1,) * 2)
)

indices

In [ ]:
try:

    indices = cluster_2d_array_slices(
        _2d_array, 0, groups=np.asarray((1,) * 4 + (0,) * 2 + (1,) * 1)
    )

    print(indices)

except ValueError as exception:

    print(exception)

In [ ]:
from nd_array.compute_empirical_p_value import compute_empirical_p_value

value = 0

random_values = np.asarray((-1, 0, 2, 4, 8))

for p_value_direction in ("less", "great"):

    print()

    print(p_value_direction)

    p_value = compute_empirical_p_value(value, random_values, p_value_direction)

    print(p_value)

In [ ]:
from nd_array.compute_empirical_p_values_and_fdrs import (
    compute_empirical_p_values_and_fdrs,
)

values = np.asarray((-1, 0, 2, 4, 8))

random_values = np.asarray(range(10))

for p_value_direction in ("less_or_great", "less", "great"):

    print()

    print(p_value_direction)

    p_values, fdrs = compute_empirical_p_values_and_fdrs(
        values, random_values, p_value_direction
    )

    print(p_values, fdrs)

In [ ]:
from nd_array.compute_nd_array_margin_of_error import compute_nd_array_margin_of_error

for nd_array in (
    np.random.normal(size=10),
    np.asarray((8,)),
    np.asarray((np.nan,)),
    np.asarray((8, np.nan)),
):

    print()

    print(nd_array)

    margin_of_error = compute_nd_array_margin_of_error(nd_array, raise_for_bad=False)

    print(margin_of_error)

In [ ]:
from nd_array.get_1d_array_unique_objects_in_order import (
    get_1d_array_unique_objects_in_order,
)

_1d_array = np.asarray((-1, 0, 2, 4, 8, -1, 0, 2, 4, 8))

unique_objects_in_order = get_1d_array_unique_objects_in_order(_1d_array)

unique_objects_in_order

In [ ]:
from nd_array.get_intersections_between_2_1d_arrays import (
    get_intersections_between_2_1d_arrays,
)

_1d_array_0 = np.asarray((-1, 0, 2, 4, 8, 4, 2, 0, -1))

_1d_array_1 = np.asarray((8, 4, 2, 0, -1, 0, 2, 4, 8))

In [ ]:
pl.offline.iplot(
    [
        dict(type="scatter", x=tuple(range(_1d_array_0.size)), y=_1d_array_0),
        dict(type="scatter", x=tuple(range(_1d_array_1.size)), y=_1d_array_1),
    ]
)

is_intesection = get_intersections_between_2_1d_arrays(_1d_array_0, _1d_array_1)

is_intesection

In [ ]:
_1d_array_0 += 1

pl.offline.iplot(
    [
        dict(type="scatter", x=tuple(range(_1d_array_0.size)), y=_1d_array_0),
        dict(type="scatter", x=tuple(range(_1d_array_1.size)), y=_1d_array_1),
    ]
)

is_intesection = get_intersections_between_2_1d_arrays(_1d_array_0, _1d_array_1)

is_intesection

In [ ]:
from nd_array.log_nd_array import log_nd_array

for nd_array in (
    np.asarray((1, 2)),
    np.asarray((0, 1, 2)),
    np.asarray((-1, 0, 1, 2)),
    np.asarray((1, 2, np.nan, np.inf, -np.inf)),
    np.asarray((0, 1, 2, np.nan, np.inf, -np.inf)),
    np.asarray((-1, 0, 1, 2, np.nan, np.inf, -np.inf)),
    np.linspace(0, 2, 11),
):

    print()

    print(nd_array)

    for shift_as_necessary_to_achieve_min_before_logging in (0, 1):

        for log_base in (
            #'2',
            "e",
            #'10',
        ):

            print()

            print(shift_as_necessary_to_achieve_min_before_logging, log_base)

            nd_array_logged = log_nd_array(
                nd_array,
                shift_as_necessary_to_achieve_min_before_logging=shift_as_necessary_to_achieve_min_before_logging,
                log_base=log_base,
                raise_for_bad=False,
            )

            print(nd_array_logged)

In [ ]:
from nd_array.make_coordinates_for_reflection import make_coordinates_for_reflection

coordinates = np.asarray(range(10))

coordinates_for_reflection = make_coordinates_for_reflection(
    coordinates, coordinates[1]
)

coordinates_for_reflection

In [ ]:
from nd_array.make_mesh_grid_coordinates_per_axis import (
    make_mesh_grid_coordinates_per_axis,
)

mesh_grid_coordinates_per_axis = make_mesh_grid_coordinates_per_axis(
    (0, 0), (8, 8), (2, 2)
)

mesh_grid_coordinates_per_axis

In [ ]:
from nd_array.nd_array_is_sorted import nd_array_is_sorted

for nd_array in (
    np.asarray((0, 1, -1)),
    np.asarray((-1, 0, 1)),
    np.asarray((1, 0, -1)),
    np.asarray((0, 0, 0)),
    np.asarray((0, 1)),
    np.asarray((1, 0)),
):

    print()

    print(nd_array)

    is_sorted = nd_array_is_sorted(nd_array)

    print(is_sorted)

In [ ]:
from nd_array.normalize_nd_array import normalize_nd_array

In [ ]:
for _1d_array in (
    np.asarray((-1, 0, 1)),
    np.asarray((-1, -np.inf, 0, np.nan, 1, np.inf)),
    np.asarray((0, 0)),
    np.asarray((1, 1)),
    np.asarray((2,)),
):

    print()

    print(_1d_array)

    for method in ("-0-", "0-1", "sum", "rank"):

        print()

        print(method)

        try:

            normalized_1d_array = normalize_nd_array(
                _1d_array, None, method, raise_for_bad=False
            )

            print(normalized_1d_array)

        except ValueError as exception:

            print(exception)

In [ ]:
for _2d_array in (
    np.asarray(((0, 1, 2), (3, 4, 5), (6, 7, 8)), dtype=float),
    np.asarray(((np.nan, 0, 1, 2), (3, np.inf, 4, 5), (6, 7, -np.inf, 8))),
):

    print()

    print(_2d_array)

    for axis in (None, 0, 1):

        for method in ("-0-", "0-1", "sum", "rank"):

            print()

            print(axis, method)

            normalized_2d_array = normalize_nd_array(
                _2d_array, axis, method, raise_for_bad=False
            )

            print(normalized_2d_array)

In [ ]:
from nd_array.shuffle_each_2d_array_slice import shuffle_each_2d_array_slice

_2d_array = np.asarray((np.asarray(range(10)), np.asarray(range(10))[::-1]))

for axis in (0, 1):

    print()

    print(axis)

    _2d_array_shuffled = shuffle_each_2d_array_slice(_2d_array, axis)

    print(_2d_array_shuffled)

In [ ]:
from plot.make_colorscale import make_colorscale
from plot.style import (
    BINARY_COLORS_RUBY_EMERALD,
    BINARY_COLORS_WHITE_BLACK,
    CATEGORICAL_COLORS,
    CONTINUOUS_COLORSCALE_FOR_MATCH,
)

for colors in (
    CATEGORICAL_COLORS,
    BINARY_COLORS_WHITE_BLACK,
    BINARY_COLORS_RUBY_EMERALD,
):

    colorscale = make_colorscale(colors=colors)

colorscale = make_colorscale(colorscale=CONTINUOUS_COLORSCALE_FOR_MATCH)

colorscale = make_colorscale(colormap="ocean")

colorscale = make_colorscale(n_category=8)

In [ ]:
from plot.make_random_color import make_random_color

make_random_color("hex")

In [ ]:
from plot.plot_bar import plot_bar

positions = (("Position 0",), ("Position 1",), ("Position 2",))

heights = ((2,), (4,), (8,))

colors = ("Red", "Green", "Blue")

for orientation, xs, ys in (("v", positions, heights), ("h", heights, positions)):

    plot_bar(
        xs,
        ys,
        names=colors,
        colors=colors,
        position_labels=colors,
        position_label_colors=colors,
        orientation=orientation,
        xaxis_dtick=2,
        yaxis_dtick=2,
    )

In [ ]:
from plot.plot_bubble_map import plot_bubble_map

n_row = 8

n_column = 16

df = pd.DataFrame(
    np.random.random_sample(size=(n_row, n_column)),
    index=tuple("Index {}".format(i) for i in range(n_row)),
    columns=tuple("Column {}".format(j) for j in range(n_column)),
)

df.iloc[0, 1] = 1.6

print(df)

plot_bubble_map(df)

In [ ]:
from plot.plot_color_text import plot_color_text

plot_color_text(("#ff0000", "#0000ff"), ("Red", "Blue"))

In [ ]:
from plot.plot_distributions import plot_distributions

x = np.random.random_sample(size=80)

plot_distributions((x, x * 2))

In [ ]:
from plot.plot_heat_map import plot_heat_map

df = pd.DataFrame([(0, 3), (2, 1)], index=("0", "1"))

plot_heat_map(df)

plot_heat_map(df.values)

df.index = ("_0", "_1")

plot_heat_map(df)

plot_heat_map(df.values)

df = pd.DataFrame(np.random.random_sample(size=(8, 16)))

indices = [0, 1]

df.iloc[indices, indices] = np.nan

df.index = ("Index {}".format(i) for i in range(df.shape[0]))

df.columns = ("Column {}".format(i) for i in range(df.shape[1]))

column_annotation = np.random.randint(0, high=df.shape[1] // 2, size=df.shape[1])

row_annotation = np.random.randint(0, high=df.shape[0] // 2, size=df.shape[0])

plot_heat_map(
    df,
    column_annotation=column_annotation,
    column_annotation_str={
        a: "Column Group {}".format(a) for a in set(column_annotation)
    },
    column_annotation_kwargs={"textangle": -90, "yshift": 88},
    row_annotation=row_annotation,
    row_annotation_str={a: "Row Group {}".format(a) for a in set(row_annotation)},
    row_annotation_kwargs={"xshift": 80},
    colorbar_x=1.16,
)

In [ ]:
from plot.plot_pie import plot_pie

n = 8

values = np.random.randint(0, 10, size=n)

labels = tuple("Label {}".format(i) for i in range(n))

annotation_color = {"A" * (i + 1): make_random_color("hex") for i in range(n)}

plot_pie(values, labels)

In [ ]:
from plot.plot_points import plot_points

x = np.linspace(-8, 8, 80)

y = np.linspace(-16, 16, 80)

plot_points((x, x), (y, y * 2))

In [ ]:
from plot.plot_table import plot_table

df = pd.DataFrame(np.random.randint(0, 1, size=(16, 8)))

df.index = ("Index {}".format(i) for i in range(df.shape[0]))

df.columns = ("Column {}".format(i) for i in range(df.shape[1]))

plot_table(df)

In [ ]:
from plot.plot_violin_or_box import plot_violin_or_box

n = 80

y = np.random.random_sample(size=n)

ys = (y, y * 2)

for violin_or_box in ("violin", "box"):

    plot_violin_or_box(ys, violin_or_box=violin_or_box)

In [ ]:
from sklearn.metrics import auc, roc_curve

In [ ]:
df = pd.read_table("titanic.tsv", index_col=0)

df = df[["sex", "age", "fare", "survived"]].dropna()

df

In [ ]:
sys.path.insert(0, "../../nd_array")

from nd_array.log_nd_array import log_nd_array
from probability.plot.plot.plot_distributions import plot_distributions

g = np.asarray(df["sex"] == "male", dtype=int)

g_name = "Gender"

a = np.asarray(df["age"])

a_name = "Age"

f = log_nd_array(
    df["fare"].values, shift_as_necessary_to_achieve_min_before_logging="0<"
)

f_name = "Fare"

s = np.asarray(df["survived"])

s_name = "Survival"

plot_distributions(
    (g, a, f, s),
    names=(g_name, a_name, f_name, s_name),
    title="Variable Distributions",
    xaxis_title="Variable Value",
)

In [ ]:
p_s1 = (s == 1).sum() / s.size

p_s1

In [ ]:
grid_size = 32

In [ ]:
from probability.infer import infer

In [ ]:
p_s__g, p_s1__g = infer((g, s), grid_size=grid_size, target=1, names=(g_name, s_name))

p_s__a, p_s1__a = infer((a, s), grid_size=grid_size, target=1, names=(a_name, s_name))

p_s__f, p_s1__f = infer((f, s), grid_size=grid_size, target=1, names=(f_name, s_name))

In [ ]:
from probability.infer_assuming_independence import infer_assuming_independence

p_s__a_f, p_s1__a_f = infer(
    (a, f, s), grid_size=grid_size, target=1, names=(a_name, f_name, s_name)
)

p_s__a_f_naive, p_s1__a_f_naive = infer_assuming_independence(
    (a, f, s), grid_size=grid_size, target=1, names=(a_name, f_name, s_name)
)

In [ ]:
from nd_array.compute_empirical_p_value import compute_empirical_p_value
from nd_array.normalize_nd_array import normalize_nd_array

maths = (
    "P(S = 1 | G)",
    "P(S = 1 | A)",
    "P(S = 1 | F)",
    "P(S = 1 | A, F)",
    "P(S = 1 | A, F) (naive)",
)

math_roc = {math: {} for math in maths}

for math, p_s1__v, vs in zip(
    maths,
    (p_s1__g, p_s1__a, p_s1__f, p_s1__a_f, p_s1__a_f_naive),
    ((g,), (a,), (f,), (a, f), (a, f)),
):

    p_s1__vv = np.full(s.size, np.nan)

    for i in range(s.size):

        coordinate = [
            [np.argmin(abs(np.linspace(v.min(), v.max(), grid_size) - v[i]))]
            for v in vs
        ]

        p_s1__vv[i] = p_s1__v[coordinate]

    fpr, tpr, t = roc_curve(s, normalize_nd_array(p_s1__vv, None, "0-1"))

    math_roc[math]["fpr"] = fpr

    math_roc[math]["tpr"] = tpr

    auc_ = auc(fpr, tpr)

    math_roc[math]["auc"] = auc_

    n_permutation_for_roc = 1000

    permuting_aucs = np.full(n_permutation_for_roc, np.nan)

    permuting_s = s.copy()

    for i in range(n_permutation_for_roc):

        np.random.shuffle(permuting_s)

        permuting_fpr, permuting_tpr, permuting_t = roc_curve(permuting_s, p_s1__vv)

        permuting_aucs[i] = auc(permuting_fpr, permuting_tpr)

    math_roc[math]["p-value"] = compute_empirical_p_value(auc_, permuting_aucs, "great")

In [ ]:
from probability.plot_bayesian_nomogram import plot_bayesian_nomogram

plot_bayesian_nomogram(
    s, 1, 0, grid_size, (p_s__g, p_s__a, p_s__f), (g_name, a_name, f_name)
)

In [ ]:
from probability.plot.plot.plot_points import plot_points

random_roc = np.linspace(0, 1, 16)

plot_points(
    (random_roc,) + tuple(math_roc[math]["fpr"] for math in maths),
    (random_roc,) + tuple(math_roc[math]["tpr"] for math in maths),
    names=("Random ROC",)
    + tuple(
        "{} | {:0.3f} | {:0.1e}".format(
            math, math_roc[math]["auc"], math_roc[math]["p-value"]
        )
        for math in maths
    ),
    modes=("markers",) + ("markers + lines",) * len(maths),
    title="ROC: G={}, A={}, F={}".format(g_name, a_name, f_name),
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    legend_orientation="h",
)

In [ ]:
training_sample_x_feature = np.asarray(((0, 1),) * 8 + ((2, 3),) * 8)

training_sample_x_feature

In [ ]:
training_sample_class = np.asarray((0,) * 8 + (1,) * 8)

training_sample_class

In [ ]:
testing_sample_x_feature = training_sample_x_feature[::-1]

testing_sample_x_feature

In [ ]:
from regression.train_and_regress import train_and_regress

testing_sample_class = train_and_regress(
    training_sample_x_feature, training_sample_class, testing_sample_x_feature
)

testing_sample_class

In [ ]:
from sequencing_process.support.support.path import clean_path

GRCH_DIRECTORY_PATH = clean_path("~/sequencing_process_grch")

assert os.path.isdir(GRCH_DIRECTORY_PATH)

PEOPLE_DIRECTORY_PATH = clean_path("~/sequencing_process_people")

assert os.path.isdir(PEOPLE_DIRECTORY_PATH)

In [ ]:
from sequencing_process.download_clinvar_vcf_gz import download_clinvar_vcf_gz

vcf_gz_file_path = download_clinvar_vcf_gz(GRCH_DIRECTORY_PATH, overwrite=True)

vcf_gz_file_path

In [ ]:
from sequencing_process.support.support.compression import gzip_decompress_file

vcf_file_path = gzip_decompress_file(vcf_gz_file_path)

vcf_file_path

In [ ]:
from sequencing_process.bgzip_and_tabix import bgzip_and_tabix

vcf_gz_file_path = bgzip_and_tabix(vcf_file_path, overwrite=True)

vcf_gz_file_path

In [ ]:
from sequencing_process.make_reference_genome import make_reference_genome

fasta_gz_file_path = make_reference_genome(GRCH_DIRECTORY_PATH, overwrite=True)

In [ ]:
from sequencing_process.simulate_sequences_using_dwgsim import (
    simulate_sequences_using_dwgsim,
)

simulate_sequences_using_dwgsim(
    fasta_gz_file_path[:-3],
    "{}/simulation".format(PEOPLE_DIRECTORY_PATH),
    n_sequence=int(1e6),
)

In [ ]:
from time import sleep

from support.multiprocess import multiprocess


def function(a, b):

    sleep(2)

    return a + b


returns = multiprocess(function, ((0, 1), (2, 3), (4, 5)), 2)

returns

In [ ]:
from support.df import drop_df_slice_greedily

df = pd.DataFrame(np.random.random_sample(size=(2, 4)))

df_dropped = drop_df_slice_greedily(df)

print(df_dropped)

df_dropped = drop_df_slice_greedily(df.T)

print(df_dropped)

In [ ]:
from tables import NoSuchNodeError

from variant.support.support.path import clean_path

In [ ]:
vcf_gz_file_path = clean_path("~/people/0/genome.vcf.gz")

assert os.path.isfile(vcf_gz_file_path)

In [ ]:
from variant.access_vcf_dict import read_vcf_gz_and_make_vcf_dict

vcf_dict = read_vcf_gz_and_make_vcf_dict(vcf_gz_file_path)

vcf_dict.keys()

In [ ]:
vcf_dict["meta_information"]

In [ ]:
vcf_dict["vcf_df"]

In [ ]:
vcf_dict["clean_vcf_df"]

In [ ]:
from variant.access_vcf import count_vcf_gz_rows

field_n = count_vcf_gz_rows(
    vcf_gz_file_path,
    info_fields_to_count=(
        "ANN",
        "CAF",
        "CLNDISDB",
        "CLNDN",
        "CLNSIG",
        "CLNREVSTAT",
        "CLNVI",
    ),
    format_fields_to_count=("GT", "AD", "DP"),
)

field_n

In [ ]:
from variant.VariantHDF5 import VariantHDF5

variant_hdf5 = VariantHDF5(vcf_gz_file_path, reset=True)

In [ ]:
try:

    variant_dict = variant_hdf5.get_variant_by_id("rs235")

    print(variant_dict)

except KeyError as exception:

    warn("(get_variant_by_id) KeyError: {}.".format(exception))

In [ ]:
try:

    variant_dict = variant_hdf5.get_variant_by_id("rs88888888")

    print(variant_dict)

except KeyError as exception:

    warn("(get_variant_by_id) KeyError: {}.".format(exception))

In [ ]:
try:

    variant_dict = variant_hdf5.get_variants_by_gene("KRAS")

    print(variant_dict)

except KeyError as exception:

    warn("(get_variants_by_gene) KeyError: {}.".format(exception))

In [ ]:
try:

    variant_dict = variant_hdf5.get_variants_by_gene("Kwat")

    print(variant_dict)

except KeyError as exception:

    warn("(get_variants_by_gene) KeyError: {}.".format(exception))

In [ ]:
try:

    variant_dict = variant_hdf5.get_variants_by_region("8", 0, 80000)

    print(variant_dict)

except NoSuchNodeError as exception:

    warn("(get_features_by_region) NoSuchNodeError: {}.".format(exception))

In [ ]:
try:

    variant_dict = variant_hdf5.get_variants_by_region("88888888", 0, 80000)

    print(variant_dict)

except NoSuchNodeError as exception:

    warn("(get_features_by_region) NoSuchNodeError: {}.".format(exception))

In [ ]:
from variant.access_maf import make_maf_from_vcf

make_maf_from_vcf(vcf_gz_file_path)